<a href="https://colab.research.google.com/github/reeshabhkumarranjan/ML-Course-Project/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import numpy as np
import glob
from keras import Sequential
import sklearn, sklearn.model_selection
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Activation
from keras.utils import to_categorical

In [2]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# loading data
colab_directory = '/content/drive/My Drive/Colab Notebooks/ML Course Project/'
x = np.load(colab_directory + "x.npy")
y = np.load(colab_directory + "y.npy")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [18]:
# LSTM model
epochs = 15
model_lstm = Sequential([
	LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2])),
	Dropout(0.5),
	Dense(200),
	Activation('relu'),
	Dense(y_train.shape[1]),
	Activation('softmax'),
])
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(x_train, y_train, epochs=epochs, batch_size=64, verbose=False)
useless, accuracy = model_lstm.evaluate(x_test, y_test, batch_size=64, verbose=False)
print("Accuracy of LSTM model on test: ", accuracy)

Accuracy of LSTM model on test:  0.8699999928474427


In [25]:
# CNN-LSTM
epochs = 15
trainX = x_train.reshape((x_train.shape[0], 3, 20, x_train.shape[2]))
testX = x_test.reshape((x_test.shape[0], 3, 20, x_test.shape[2]))

conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu')
model_lstm_cnn = Sequential([
	TimeDistributed(conv1, input_shape=(None, 20, x_train.shape[2])),
	TimeDistributed(conv2),
	TimeDistributed(Dropout(0.5)),
	TimeDistributed(MaxPooling1D(pool_size=2)),
	TimeDistributed(Flatten()),
	LSTM(200),
	Dropout(0.5),
	Dense(200),
	Activation('relu'),
	Dense(y_train.shape[1]),
	Activation('softmax')
])
model_lstm_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm_cnn.fit(trainX, y_train, epochs=epochs, batch_size=64, verbose=False)
useless, accuracy = model_lstm_cnn.evaluate(testX, y_test, batch_size=64, verbose=False)
print("Accuracy on test-set for LSTM-CNN: ", accuracy)

Accuracy on test-set for LSTM-CNN:  0.9299999976158142


In [27]:
# LSTM + conv
epochs = 15
trainX = x_train.reshape((x_train.shape[0], 3, 1, 20, x_train.shape[2]))
testX = x_test.reshape((x_test.shape[0], 3, 1, 20, x_test.shape[2]))

model = Sequential([
	ConvLSTM2D(filters=100, kernel_size=(1,3), input_shape=(3, 1, 20, x_train.shape[2])),
	Activation('relu'),
	Dropout(0.5),
	Flatten(),
	Dense(200),
	Activation('relu'),
	Dense(y_train.shape[1]),
	Activation('softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, y_train, epochs=epochs, batch_size=64, verbose=False)
useless, accuracy = model.evaluate(testX, y_test, batch_size=64, verbose=False)
print("Accuracy on test for LSTM-Conv: ", accuracy)

Accuracy on test for LSTM-Conv:  0.9300000023841858
